In [107]:
import pandas as pd
from sklearn.cluster import KMeans
from feature_engine.encoding import OneHotEncoder as fe_OneHotEncoder
from feature_engine.encoding import CountFrequencyEncoder
from feature_engine.encoding import OrdinalEncoder
from sklearn.pipeline import Pipeline

In [108]:
df = pd.read_csv('salaries.csv')

In [109]:
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2022,SE,FT,Marketing Data Analyst,200000,USD,200000,GB,100,GB,S
1,2022,EN,FT,Data Scientist,96000,CAD,74378,CA,100,CA,L
2,2022,SE,FT,Data Science Lead,165000,USD,165000,US,50,US,S
3,2022,EN,FT,Data Scientist,27000,GBP,33599,GB,50,GB,L
4,2022,SE,FT,Data Engineer,185900,USD,185900,US,0,US,M


In [110]:
df.experience_level.unique()

array(['SE', 'EN', 'MI', 'EX'], dtype=object)

In [111]:
mapping_experience_level = {'EN': 1,'MI': 2, 'SE': 3, 'EX': 4}
mapping_company_size = {'S': 1, 'M': 2, 'L': 3}
df = df.replace({'company_size': mapping_company_size, 'experience_level': mapping_experience_level})  # Lable encoding

encodingPipeline = Pipeline([  # Define the pipeline
    #('ordinalEncoder', OrdinalEncoder(encoding_method='arbitrary', variables=["company_size"])),
    ('oneHotEnc', fe_OneHotEncoder(top_categories=None, variables=["employment_type", "salary_currency"], drop_last=True)),
    ('countEncoder', CountFrequencyEncoder(encoding_method='count', variables=["job_title", "employee_residence", "company_location"]))
    
])

encodingPipeline.fit(df)  # Fit the pipeline
df = encodingPipeline.transform(df)  # Save imputed data to original df

In [112]:
df.head()

,work_year,experience_level,job_title,salary,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,employment_type_FT,...,salary_currency_BRL,salary_currency_DKK,salary_currency_SGD,salary_currency_CNY,salary_currency_PLN,salary_currency_CHF,salary_currency_JPY,salary_currency_HUF,salary_currency_MXN,salary_currency_TRY
0,2022,3,2,200000,200000,78,100,82,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2022,1,310,96000,74378,36,100,37,3,1,...,0,0,0,0,0,0,0,0,0,0
2,2022,3,1,165000,165000,795,50,823,1,1,...,0,0,0,0,0,0,0,0,0,0
3,2022,1,310,27000,33599,78,50,82,3,1,...,0,0,0,0,0,0,0,0,0,0
4,2022,3,276,185900,185900,795,0,823,2,1,...,0,0,0,0,0,0,0,0,0,0


In [113]:
k_means = KMeans(n_clusters=3)
k_means.fit(df)

labels = k_means.predict(df)

c:\Users\leand\anaconda3\envs\cds106\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=5.
  warnings.warn(
